# Create, update, and manage image data in BigQuery

In [26]:
! pip install --upgrade google-cloud-bigquery google-cloud-storage jsonlines pandas

     |████████████████████████████████| 11.3 MB 5.2 MB/s eta 0:00:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.2
    Uninstalling pandas-1.3.2:
      Successfully uninstalled pandas-1.3.2


In [2]:
## COLAB ONLY! ##
try:
    from google.colab import auth
    auth.authenticate_user()
except:
    print('Not running in Colab')

Not running in Colab


In [34]:
#@markdown Store the resource names for this notebook

bigquery_table_id = 'fantasy_maps.dataset_maps'
display_name = 'dnd-maps-bigquery'  #@param {type: "string"}
PROJECT_ID = 'video-erschmid' #@param {type: "string"}
LOCATION = 'us-central1' #@param {type: "string"}
gcs_bucket_name = 'video-erschmid' #@param {type: "string"}

In [20]:
!gcloud config get-value project
!gcloud config set project $PROJECT_ID
!gcloud config get-value project

video-erschmid
Updated property [core/project].
video-erschmid


## Populate the BigQuery table

Start with a list of the training data and batch prediction data.

In [42]:
# Compile a list of JSONL sources to populate table with
# ONE-TIME / FIRST TIME thing

training_data = 'DnD/map_training_data.jsonl'
batch_predict_data = 'DnD-batch-predict-input/20211005225454/output/prediction-dnd-maps-full-grid-model-online-20210913212620-2021-10-05T22:55:16.624758Z/predictions_00001.jsonl'

### Upload the original training data

Download `training_data` file from Storage.

In [16]:
from google.cloud import storage

storage_client = storage.Client(project=PROJECT_ID)

bucket = storage_client.bucket(gcs_bucket_name)
training_data_blob = bucket.get_blob(training_data)
blob_bytes = training_data_blob.download_as_bytes()
training_data_str = blob_bytes.decode('utf-8')

Convert training data to Pandas `DataFrame` object

In [29]:
training_data_rows = training_data_str.split('\n')
json.loads(training_data_rows[0])

{'imageGcsUri': 'gs://video-erschmid/DnD/GL_OasisCity_Rain.jpg',
 'boundingBoxAnnotations': [{'xMin': 0.020972644376899698,
   'yMin': 0.014935064935064935,
   'yMax': 0.03051948051948052,
   'xMax': 0.04285714285714286,
   'displayName': 'cell'},
  {'xMin': 0.020972644376899698,
   'yMin': 0.030086580086580085,
   'yMax': 0.04567099567099567,
   'xMax': 0.04285714285714286,
   'displayName': 'cell'},
  {'xMin': 0.020972644376899698,
   'yMin': 0.04523809523809524,
   'yMax': 0.06082251082251082,
   'xMax': 0.04285714285714286,
   'displayName': 'cell'},
  {'xMin': 0.020972644376899698,
   'yMin': 0.06038961038961039,
   'yMax': 0.07597402597402597,
   'xMax': 0.04285714285714286,
   'displayName': 'cell'},
  {'xMin': 0.020972644376899698,
   'yMin': 0.07554112554112555,
   'yMax': 0.09112554112554112,
   'xMax': 0.04285714285714286,
   'displayName': 'cell'},
  {'xMin': 0.020972644376899698,
   'yMin': 0.09069264069264069,
   'yMax': 0.10627705627705628,
   'xMax': 0.04285714285714286

In [31]:
import pandas as pd
import json

# List-row format: image_uri, training_data, source, ID 
columns = ['image_uri', 'training_data', 'source', 'ID']

training_data_list = []

for row in training_data_rows:
    try:
        row_json = json.loads(row)
        image_gcs_uri = row_json['imageGcsUri']
        image_name = image_gcs_uri.split('/')[-1]
        image_id = f"original.{image_name.lower()}"
        source = "manual"

        training_data_list.append([image_gcs_uri, row, source, image_id])
    except:
        continue
    
df = pd.DataFrame(training_data_list, columns=columns)

df.head(2)

,image_uri,training_data,source,ID
0,gs://video-erschmid/DnD/GL_OasisCity_Rain.jpg,"{""imageGcsUri"": ""gs://video-erschmid/DnD/GL_Oa...",manual,original.gl_oasiscity_rain.jpg
1,gs://video-erschmid/DnD/G_AbandonedMineEntranc...,"{""imageGcsUri"": ""gs://video-erschmid/DnD/G_Aba...",manual,original.g_abandonedmineentrance_crystal.jpg


In [35]:
from google.cloud import bigquery

bigquery_client = bigquery.Client()

job_config = job_config = bigquery.LoadJobConfig(schema=[
    bigquery.SchemaField("image_uri", "STRING"),
    bigquery.SchemaField("training_data", "STRING"),
    bigquery.SchemaField("source", "STRING"),
    bigquery.SchemaField("ID", "STRING"),
])

job = bigquery_client.load_table_from_dataframe(
    df, bigquery_table_id, job_config=job_config
)

job.result()

### Upload last batch prediction results

In [43]:
batch_prediction_data_blob = bucket.get_blob(batch_predict_data)
bp_blob_bytes = batch_prediction_data_blob.download_as_bytes()
bp_str = bp_blob_bytes.decode('utf-8')

In [46]:
bp_data_rows = bp_str.split('\n')
#print(bp_data_rows[0])
json.loads(bp_data_rows[0])

{'instance': {'content': 'gs://video-erschmid/DnD-batch-predict-input/20210930231618/mountain_outpost_[battlemap][oc][22x33][1540x2310].jpg',
  'mimeType': 'image/jpeg'},
 'prediction': {'ids': ['1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
   '1828851775340281856',
  